# 📈 ETH Network Forecasting with Auto ARIMA

This notebook forecasts Ethereum network-level metrics (tx count, fees, active addresses) using auto_arima (pmdarima). Results are saved to `results/eth/`.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from pmdarima import auto_arima

# Setup

In [2]:
os.makedirs("../../results/eth", exist_ok=True)

# Load ETH network data

In [3]:
df = pd.read_csv("../../data/eth/eth_network_daily.csv")
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df = df.asfreq('D')  # enforce daily frequency

# Optionally drop final row if suspiciously low (incomplete day)

In [4]:
if df.iloc[-1]['tx_count'] < 250 or df.iloc[-1]['active_addresses'] < 500:
    df = df.iloc[:-1]

# Forecast horizon

In [5]:
forecast_horizon = 14

# Loop over each feature

In [6]:
for col in ['tx_count', 'total_fees', 'active_addresses']:
    ts = df[col].dropna()

    # Auto ARIMA model
    model = auto_arima(ts, seasonal=False, stepwise=True, suppress_warnings=True, error_action='ignore')
    forecast = model.predict(n_periods=forecast_horizon)
    forecast_index = pd.date_range(start=ts.index[-1] + pd.Timedelta(days=1), periods=forecast_horizon, freq='D')
    forecast_series = pd.Series(forecast, index=forecast_index, name='forecast')

    # Save forecast to CSV
    forecast_df = pd.DataFrame({
        'Date': forecast_index,
        f'{col}_forecast': forecast
    })
    forecast_df.to_csv(f"../../results/eth/{col}_forecast.csv", index=False)

    # Plot historical + forecast
    plt.figure(figsize=(12, 4))
    plt.plot(ts.index, ts.values, label='Historical')
    plt.plot(forecast_index, forecast, label='Forecast', linestyle='--')
    plt.title(f"{col.replace('_', ' ').title()} Forecast (ETH)")
    plt.xlabel("Date")
    plt.ylabel(col.replace('_', ' ').title())
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"../../results/eth/{col}_forecast_plot.png")
    plt.close()

/opt/anaconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/

In [7]:
print("ETH forecasting completed and saved to `results/eth/`.")

ETH forecasting completed and saved to `results/eth/`.
